<a href="https://colab.research.google.com/github/mkazachkin/LearningTransfer/blob/main/HW_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание

Реализуйте классификацию эмоциональной окрашенности текстов при помощи объекта pipeline.

1. Найдите тип задач Sentiment Analysis на huggingface.co
2. Найдите модель для русского языка (примеры: rubert-tiny2..., rubert-base...)

In [6]:
from transformers import pipeline

pipe = pipeline("text-classification", model="seara/rubert-base-cased-ru-go-emotions")

In [7]:
labels = {
    'admiration': 'восхищение', 'amusement': 'веселье', 'anger': 'злость', 'annoyance': 'раздражение',
    'approval': 'одобрение', 'caring': 'забота', 'confusion': 'непонимание', 'curiosity': 'любопытство',
    'desire': 'желание', 'disappointment': 'разочарование', 'disapproval': 'неодобрение', 'disgust': 'отвращение',
    'embarrassment': 'смущение', 'excitement': 'возбуждение', 'fear': 'страх', 'gratitude': 'признательность',
    'grief': 'горе', 'joy': 'радость', 'love': 'любовь', 'nervousness': 'нервозность', 'optimism': 'оптимизм',
    'pride': 'гордость', 'realization': 'осознание', 'relief': 'облегчение', 'remorse': 'раскаяние',
    'sadness': 'грусть', 'surprise': 'удивление', 'neutral': 'нейтральность'
}


def get_label(text, labels, model):
    predictions = model(text)
    label = predictions[0]['label']
    score = predictions[0]['score']
    return labels[label], round(score, 2)


def classifier(text_list):
    results = []
    for text in text_list:
        result = get_label(text, labels, pipe)
        results.append(result)
    return results

In [8]:
classifier(["Отличный товар",
            "Мне не понравилось",
            "Прекрасно выглядит, ничего не меняйте!",
            "Ничего не понял",
            "У меня не работала розетка, в остальном все отлично"])

[('восхищение', 0.92),
 ('нейтральность', 0.41),
 ('восхищение', 0.94),
 ('нейтральность', 0.42),
 ('веселье', 0.4)]